In [ ]:
from pyclim_noresm.aerosol_forcing import calc_total_ERF_TOA, merge_exp_ctrl
from workflow.scripts.utils import load_CMIP_data, copy_meta_data_CMIP, regrid_global
from pyclim_noresm.general_util_funcs import yearly_avg, global_avg
import time
import xarray as xr
import numpy as np

In [ ]:
VARS = snakemake.config['variables']

vName_dw_SW = VARS[snakemake.wildcards.vName][1]
vName_up_SW = VARS[snakemake.wildcards.vName][0]
vName_up_LW = VARS[snakemake.wildcards.vName][2]
exp_dw_SW = load_CMIP_data(snakemake.input.exp_downwelling_SW, data_vars=[vName_dw_SW])
exp_up_SW = load_CMIP_data(snakemake.input.exp_upwelling_SW, data_vars=[vName_up_SW])
exp_up_LW = load_CMIP_data(snakemake.input.exp_upwelling_LW, data_vars=[vName_up_LW])
ctrl_dw_SW = load_CMIP_data(snakemake.input.ctrl_downwelling_SW,data_vars=[vName_dw_SW])
ctrl_up_SW = load_CMIP_data(snakemake.input.ctrl_upwelling_SW, data_vars=[vName_up_SW])
ctrl_up_LW = load_CMIP_data(snakemake.input.ctrl_upwelling_LW,data_vars=[vName_up_LW])
dw_SW = merge_exp_ctrl(exp_dw_SW, ctrl_dw_SW)
up_LW = merge_exp_ctrl(exp_up_LW, ctrl_up_LW)
up_SW = merge_exp_ctrl(exp_up_SW, ctrl_up_SW)

In [ ]:
if vName_up_SW in ['rsutaf', 'rsutafcs'] and snakemake.wildcards.model in ['NorESM2-LM','NorESM2.0.6dev-LM']:
    up_SW[vName_up_SW] = np.abs(up_SW[vName_up_SW]-dw_SW[vName_dw_SW])
    up_SW[f'control_{vName_up_SW}'] = np.abs(up_SW[f'control_{vName_up_SW}']-dw_SW[f'control_{vName_dw_SW}'])
ERF = calc_total_ERF_TOA(dw_SW[vName_dw_SW], up_SW[vName_up_SW],
                up_LW[vName_up_LW],
                dw_SW[f'control_{vName_dw_SW}'].rename(vName_dw_SW)
                , up_SW[f'control_{vName_up_SW}'].rename(vName_up_SW),
                up_LW[f'control_{vName_up_LW}'].rename(vName_up_LW))


if snakemake.wildcards.freq == 'Ayear':
    ERF = yearly_avg(ERF)



ERF = ERF.to_dataset(name=snakemake.wildcards.vName)

if snakemake.config['regrid_params'].get('regrid_params') and snakemake.config.get('regrid', True):
    if not ERF.cf.bounds.get('lon', None):
        ERF = ERF.cf.add_bounds(['lon', 'lat'])
    grid_params = snakemake.config['regrid_params']
    grid_path = grid_params.get('grid_path', None)
    method = grid_params.get('method', 'conservative')
    if grid_path:
        out_grid = xr.open_dataset(grid_path)
        ERF = regrid_global(ERF, out_grid, method=method)
    elif grid_params.get('dxdy',None):
        dxdy = grid_params['dxdy']
        ERF = regrid_global(ERF, lon=dxdy[0], lat=dxdy[1], method=method)

    regridded='True'
else:
    regridded='False'
attrs = copy_meta_data_CMIP(exp_dw_SW.attrs)

In [ ]:
ERF = ERF.assign_attrs(**attrs)
ERF = ERF.assign_attrs(variable_id=snakemake.wildcards.vName)
ERF.attrs['source'] = ', '.join(snakemake.input)
ERF.attrs['history'] = f'@{time.ctime()} Generated by: {snakemake.rule}'
ERF.attrs['frequency'] = snakemake.wildcards.freq
ERF.attrs['title'] = 'Effective radiative forcing TOA'
ERF.attrs['regridded'] = regridded
ERF.to_netcdf(snakemake.output.outpath)